Use the environment found on GCP at `conda activate /home/jupyter/analysis/dis2p_trials/scfair_20231003/scfair_dev_20231003_env`

In [12]:
# enable autoreload
%load_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext autoreload

import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION']='python'

import scvi
scvi.settings.seed = 0
import scanpy as sc
import anndata as ad
import torch
import numpy as np
import pandas as pd
import json
from datetime import datetime
from scipy.sparse import csr_matrix
torch.set_float32_matmul_precision('medium')
import warnings
warnings.simplefilter("ignore", UserWarning)
from sklearn.model_selection import train_test_split
import random

# import from scfair-reproducibility
from scfair_reproducibility.benchmark_VAE.vi import VI
from scfair_reproducibility.evaluation.metrics import *

# import from scib_metrics
from scfair_reproducibility.scib_metrics_dev.src.scib_metrics.benchmark import Benchmarker

# other methods
# import cpa
# from data.biolord_dev.src import biolord

Global seed set to 0


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
#os.environ["CUDA_VISIBLE_DEVICES"]=""

In [14]:
from dis2pm.dis2pmvae import *
from dis2pm.dis2pmvi import *
from dis2pm.ood_m import *
from dis2pm.trainingplan_m import *
from dis2pm.utils_m import *

# prepare data: Heart Atlas

Define function to convert covariate names to numerical values

In [15]:
def create_cats_idx(adata, cats):
    # create numerical index for each attr in cats

    for i in range(len(cats)):
        values = list(set(adata.obs[cats[i]]))

        val_to_idx = {v: values.index(v) for v in values}

        idx_list = [val_to_idx[v] for v in adata.obs[cats[i]]]

        adata.obs[cats[i] + '_idx'] = pd.Categorical(idx_list)

    return adata

Load and subsample the heart atlas dataset from Litvinukova et al. Nature 2020

In [16]:
adata = scvi.data.heart_cell_atlas_subsampled()

# preprocess dataset
sc.pp.filter_genes(adata, min_counts=3)
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=1200,
    subset=True,
    layer="counts",
    flavor="seurat_v3",
)

print(adata)
print(adata[:,1198].X)
adata_atac = adata[:, -200:].copy()
adata_rna = adata[:, :1000].copy()
adata_obs = adata.obs.copy()
adata_uns = adata.uns.copy()
adata_atac.X.data = np.ones(adata_atac.X.data.shape)
adata = ad.concat([adata_rna, adata_atac], axis=1)
adata.obs = adata_obs
adata.uns = adata_uns
print(adata[:,:].X.sum(axis=0))
print(adata)

# specify name of dataset 
data_name = 'HeartAtlas'

# specify attributes
cats = ['cell_type', 'cell_source', 'gender', 'region']

# create numerical index for each attr in cats
create_cats_idx(adata, cats)

# save adata
# adata.write_h5ad('data/heart_preprocessed1200.h5ad')

today = datetime.today().strftime('%Y-%m-%d')

INFO     File data/hca_subsampled_20k.h5ad already downloaded                                                      


/home/jupyter/analysis/dis2p_trials/scfair_20231003/scfair_dev_20231003_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/jupyter/analysis/dis2p_trials/scfair_20231003/scfair_dev_20231003_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


AnnData object with n_obs × n_vars = 18641 × 1200
    obs: 'NRP', 'age_group', 'cell_source', 'cell_type', 'donor', 'gender', 'n_counts', 'n_genes', 'percent_mito', 'percent_ribo', 'region', 'sample', 'scrublet_score', 'source', 'type', 'version', 'cell_states', 'Used'
    var: 'gene_ids-Harvard-Nuclei', 'feature_types-Harvard-Nuclei', 'gene_ids-Sanger-Nuclei', 'feature_types-Sanger-Nuclei', 'gene_ids-Sanger-Cells', 'feature_types-Sanger-Cells', 'gene_ids-Sanger-CD45', 'feature_types-Sanger-CD45', 'n_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'cell_type_colors', 'log1p', 'hvg'
    layers: 'counts'
  (0, 0)	4.5277414
  (3, 0)	4.0320454
  (4, 0)	4.196429
  (7, 0)	2.1310735
  (9, 0)	1.2068739
  (11, 0)	2.9385843
  (14, 0)	3.3909585
  (15, 0)	4.0477786
  (16, 0)	5.345359
  (18, 0)	1.5867883
  (20, 0)	1.2203416
  (21, 0)	4.835906
  (24, 0)	2.8654125
  (25, 0)	2.3536403
  (28, 0)	2.6903114
  (32, 0)	1.9224532
  (35, 0)	2.4982107
  (36,

/home/jupyter/analysis/dis2p_trials/scfair_20231003/scfair_dev_20231003_env/lib/python3.10/site-packages/anndata/_core/merge.py:217: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(dtype):
/home/jupyter/analysis/dis2p_trials/scfair_20231003/scfair_dev_20231003_env/lib/python3.10/site-packages/anndata/_core/merge.py:217: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(dtype):


[[5590.68177193 1019.82674909 1078.93260568 ... 7484.
  7558.         8864.        ]]
AnnData object with n_obs × n_vars = 18641 × 1200
    obs: 'NRP', 'age_group', 'cell_source', 'cell_type', 'donor', 'gender', 'n_counts', 'n_genes', 'percent_mito', 'percent_ribo', 'region', 'sample', 'scrublet_score', 'source', 'type', 'version', 'cell_states', 'Used'
    var: 'gene_ids-Harvard-Nuclei', 'feature_types-Harvard-Nuclei', 'gene_ids-Sanger-Nuclei', 'feature_types-Sanger-Nuclei', 'gene_ids-Sanger-Cells', 'feature_types-Sanger-Cells', 'gene_ids-Sanger-CD45', 'feature_types-Sanger-CD45', 'n_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'cell_type_colors', 'log1p', 'hvg'
    layers: 'counts'


/home/jupyter/analysis/dis2p_trials/scfair_20231003/scfair_dev_20231003_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [17]:
# df = adata.to_df()
# print(df)
# df_subset = df.iloc[:, -200:]
# print(df_subset)
# print(df_subset[df_subset.ge(2).any(axis=1)])



# train model: dis2p(n_cf=1)

In [18]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [21]:
# train params
epochs = 50 #400
batch_size = 7 #128
cf_weight = 1
beta = 1
clf_weight = 50
adv_clf_weight = 10
adv_period = 1
n_cf = 1

# architecture params
n_layers=1

train_dict = {'max_epochs': epochs, 'batch_size': batch_size, 'cf_weight': cf_weight,
              'beta': beta, 'clf_weight': clf_weight, 'adv_clf_weight': adv_clf_weight,
              'adv_period': adv_period, 'n_cf': n_cf}

module_name = 'dis2pm'
pre_path = f'models/{module_name}'
if not os.path.exists(pre_path):
    os.makedirs(pre_path)

# specify a name for your model
model_name =  f'{today},{module_name},{data_name},' + f'n_layers={n_layers},' + ','.join(k + '=' + str(v) for k, v in train_dict.items())

# load model (if trained before)
# try:
#     model = Dis2pmVI.load(f"{pre_path}/{model_name}", adata=adata)

# # trains the model (if not trained before) and save it into: pre_path + model_name
# except:

Dis2pmVI.setup_anndata(
    adata,
    layer='counts',
    categorical_covariate_keys=cats,
    continuous_covariate_keys=[]
)
print(adata)
model = Dis2pmVI(adata, n_genes=1000, n_regions=200, n_layers=n_layers)
model.train( **train_dict, )
model.save(f"{pre_path}/{model_name}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/jupyter/analysis/dis2p_trials/scfair_20231003/scfair_dev_20231003_env/lib/python3.10/site-packages/lightning/pytorch/core/optimizer.py:321: RuntimeWarning: The lr scheduler dict contains the key(s) ['monitor'], but the keys will be ignored. You need to call `lr_scheduler.step()` manually in manual optimization.
  rank_zero_warn(


AnnData object with n_obs × n_vars = 18641 × 1200
    obs: 'NRP', 'age_group', 'cell_source', 'cell_type', 'donor', 'gender', 'n_counts', 'n_genes', 'percent_mito', 'percent_ribo', 'region', 'sample', 'scrublet_score', 'source', 'type', 'version', 'cell_states', 'Used', 'cell_type_idx', 'cell_source_idx', 'gender_idx', 'region_idx', '_scvi_batch', '_scvi_labels'
    var: 'gene_ids-Harvard-Nuclei', 'feature_types-Harvard-Nuclei', 'gene_ids-Sanger-Nuclei', 'feature_types-Sanger-Nuclei', 'gene_ids-Sanger-Cells', 'feature_types-Sanger-Cells', 'gene_ids-Sanger-CD45', 'feature_types-Sanger-CD45', 'n_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'cell_type_colors', 'log1p', 'hvg', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: '_scvi_extra_categorical_covs'
    layers: 'counts'
1000
debug4
n_cats_per_cov is (11, 4, 2, 6)
debug4
debug5
200
Epoch 1/50:   0%|          | 0/50 [00:00<?, ?it/s]the p size is torch.Size([7, 200])
the x size is torch

In [17]:
# n_layers=1
# epochs = 50 #400
# batch_size = 4 #128
# cf_weight = 1
# beta = 1
# clf_weight = 50
# adv_clf_weight = 10
# adv_period = 1
# n_cf = 1
# n_genes=1000
# n_regions=200

n_latent_shared = 10
n_input_regions = 200
n_layers= 1 
n_hidden=128 
deeply_inject_covariates=True,
use_batch_norm=True,
use_layer_norm=False,
n_cats_per_cov = (11, 4, 2, 6)

x_decoder_acc =  DecoderPeakVI(
                    n_latent_shared,
                    n_input_regions,
                    n_cat_list=n_cats_per_cov,
                    n_layers=n_layers,
                    n_hidden=n_hidden,
                    deep_inject_covariates=deeply_inject_covariates,
                    use_batch_norm=use_batch_norm,
                    use_layer_norm=use_layer_norm,
                )
    
idx=1
x_=torch.tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  2.,
          1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
          0.,  4.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
          0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  2.,
          6.,  2.,  1.,  5.],
        [ 0.,  0.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
          1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  2.,  0.,  0.,  0.,  1.,
          0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
          0.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  2.,  0.,  0.,  0.,  0.,  0.,
          1.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          3.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,
          0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,
          0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,
          0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0., 10., 26., 38.,
         34., 22., 14., 23.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.]] )


cat_cov_ = torch.tensor( [[8., 0., 1., 2.],
        [2., 2., 1., 1.],
        [2., 0., 1., 2.],
        [8., 3., 0., 4.]] )
detach_x=False
detach_z=False


Dis2pmVI.setup_anndata(
    adata,
    layer='counts',
    categorical_covariate_keys=cats,
    continuous_covariate_keys=[]
)

# model = Dis2pmVI(adata, n_genes=1000, n_regions=200, n_layers=n_layers)

model._module_cls.sub_forward_acc(idx=idx + 1, x=x_, cat_covs=cat_cov_)
#model.Dis2pmVAE.sub_forward_acc(idx=idx + 1, x=x_, cat_covs=cat_cov_)


n_cats_per_cov = (
    self.adata_manager.get_state_registry(
        REGISTRY_KEYS.CAT_COVS_KEY
    ).n_cats_per_key
    if REGISTRY_KEYS.CAT_COVS_KEY in self.adata_manager.data_registry
    else None
)
# self.module = self._module_cls(
#     #n_input_genes=self.summary_stats.n_vars,
#     n_input_genes=n_genes,
#     n_input_regions=n_regions,
#     n_cats_per_cov=n_cats_per_cov,
#     n_hidden=n_hidden,
#     n_latent_shared=n_latent_shared,
#     n_latent_attribute=n_latent_attribute,
#     n_layers=n_layers,
#     dropout_rate=dropout_rate,
#     gene_likelihood=gene_likelihood,
#     latent_distribution=latent_distribution,
#     **model_kwargs,
# # )


# model1 = Dis2pmVAE(
#         n_input_genes=n_genes,
#     n_input_regions=n_regions,
#     n_cats_per_cov=n_cats_per_cov,
#     n_hidden=n_hidden,
#     n_latent_shared=n_latent_shared,
#     n_latent_attribute=n_latent_attribute,
#     n_layers=n_layers,
#     dropout_rate=dropout_rate,
#     gene_likelihood=gene_likelihood,
#     latent_distribution=latent_distribution,
#     **model_kwargs,
# )
    
cat_covs =cat_cov_
if detach_x:
    x_ = x.detach()

library = torch.ones(x_.size(dim=0) ) 

cat_in = torch.split(cat_covs, 1, dim=1)

qz, z = (self.z_encoders_list_acc[idx](x_, *cat_in))
if detach_z:
    z = z.detach()

dec_cats = [cat_in[j] for j in range(len(cat_in)) if j != idx-1]

x_decoder = x_decoder_acc

px_scale, px_r, px_rate, px_dropout = x_decoder(
    self.dispersion,
    z,
    library,
    *dec_cats
)     

TypeError: Dis2pmVAE.sub_forward_acc() missing 1 required positional argument: 'self'

# get latent space

In [ ]:
# load model
pre_path = f'models/dis2p'
model_name = f'{today},dis2p,HeartAtlas,n_layers=1,max_epochs=400,batch_size=128,cf_weight=1,beta=1,clf_weight=50,adv_clf_weight=10,adv_period=1,n_cf=1'
model = Dis2pVI.load(f"{pre_path}/{model_name}", adata=adata)

# Z_0
adata.obsm[f'dis2p_Z_0'] = model.get_latent_representation(nullify_cat_covs_indices=[s for s in range(len(cats))], nullify_shared=False)

for i in range(len(cats)):
    null_idx = [s for s in range(len(cats)) if s != i]
    # Z_i
    adata.obsm[f'dis2p_Z_{i+1}'] = model.get_latent_representation(nullify_cat_covs_indices=null_idx, nullify_shared=True)
    # Z_{-i}
    adata.obsm[f'dis2p_Z_not_{i+1}'] = model.get_latent_representation(nullify_cat_covs_indices=[i], nullify_shared=False)

# plot UMAP latent

In [ ]:
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

In [ ]:
for i in range(len(cats) + 1):  # loop over all Z_i

    latent_name = f'dis2p_Z_{i}'

    print(f"---UMAP for {latent_name}---")

    sc.pp.neighbors(adata, use_rep=f"{latent_name}")
    sc.tl.umap(adata)

    sc.pl.umap(
        adata,
        color=cats,
        ncols=len(cats),
        frameon=False,
    )